In [1]:
name = "CombineSignals"
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [33]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
alphas = [
    ["UserItemBiases"]
    ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
    ["ItemCFResid.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
    ["MatrixFactorization.$K" for K in downcast_to_int.([10, 20, 40])]
    ["ItemCFRelated.$name" for name in ["all", "strict"]] #TODO erase strict
    # ["MatrixFactorization$K" for K in downcast_to_int.([10, 20, 40])] # TODO erase
    # ["ItemCFEmbed.64"]
]

14-element Vector{String}:
 "UserItemBiases"
 "ItemCF.16"
 "ItemCF.64"
 "ItemCF.256"
 "ItemCF.1024"
 "ItemCFResid.16"
 "ItemCFResid.64"
 "ItemCFResid.256"
 "ItemCFResid.1024"
 "MatrixFactorization.10"
 "MatrixFactorization.20"
 "MatrixFactorization.40"
 "ItemCFRelated.all"
 "ItemCFRelated.strict"

In [34]:
function evaluate(X, y, β)
    y_pred = X * β
    #y_pred = clamp.(y_pred, 1, 10)
    rmse(y, y_pred), mae(y, y_pred), r2(y, y_pred), mse(y, y_pred)
end;

In [35]:
function get_dep(split)
    return get_split(split).rating
end

function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(length(users), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [36]:
β = get_indep("validation", alphas) \ get_dep("validation")

14-element Vector{Float64}:
  0.9915592088574635
  0.13126232057409348
  0.08674694545884268
 -0.014685501923813056
  0.42729225302206725
  0.01513246603960907
  0.021482683996924004
  0.15796650553064512
  1.6324035662645906
  0.15327661142243515
  0.1466221116911372
  0.15013298700691638
  2.645529810275075
  1.877112540771867

In [37]:
evaluate(get_indep("validation", alphas), get_dep("validation"), β)

(1.093879907684041, 0.7919769463300685, 0.6225990640379585, 1.1965732524348465)

In [38]:
evaluate(get_indep("test", alphas), get_dep("test"), β)

(1.0940811908636607, 0.7921469812597033, 0.6225279534341472, 1.1970136522016461)

In [39]:
1.1965732524348465 * 0.995

1.1905903861726723

In [9]:
write_params(Dict("β" => β, "alphas" => alphas));